In [1]:
import cv2
from keras.models import load_model,Sequential
from keras.preprocessing import image
import numpy as np
import time
import random
from datetime import datetime
from keras.models import model_from_json
import urllib.request
import requests
from mega import Mega



Using TensorFlow backend.


In [2]:
new_model = load_model("trained_CNN_model_4.h5")
mega = Mega()
m=mega.login("p1projectteam66@gmail.com","nUHDciga2p4LfXy")




In [3]:
classes_labels = ['safe_driving',
                  'texting_left',
                  'talking_on_the_phone_left',
                  'texting_right','talking_on_the_phone_right',
                  'operating_the_radio','drinking',
                  'reaching_behind',
                  'hair_and_makeup',
                  'talking_to_passenger']


In [4]:
print(classes_labels)

['safe_driving', 'texting_left', 'talking_on_the_phone_left', 'texting_right', 'talking_on_the_phone_right', 'operating_the_radio', 'drinking', 'reaching_behind', 'hair_and_makeup', 'talking_to_passenger']


In [5]:
def img_2_link(file_name):
    folder=m.find('Dri1')  #finds the drivers folder
    img=m.upload(file_name,folder[0]) #uploads image to the above folder
    link=m.get_upload_link(img) #returns link
    da=""
    for i in range(21,len(link)):
        da+=str(link[i])
    return da 

In [6]:
def upload_data(name,gps,off,image):

    try:
        URL= 'https://api.thingspeak.com/update?api_key=WB2WHZFU2S47FXL2'
        header='&field1={}&field2={}&field3={}&field4={}'.format(name,gps,off,image)
        
        new_url=URL+header
        
        data=urllib.request.urlopen(new_url)
        print('ya')
        #time.sleep(15)
        print(name,gps,off,image,"Uploaded")
    except:
        print("Failed to Upload")
   

In [7]:
def driver():
    # initialize the video stream, pointer to output video file, and
    # frame dimensions
    Driver_id = 'DRI_2'
    vs = cv2.VideoCapture("C:/Users/karth/Desktop/project/RHD.wmv")
    writer = None
    (W, H) = (None, None)
    # loop over frames from the video file stream


    predictions = []
    gps_full = []
    text_full = []
    clean_image = []
    flag = []
    gps = "1.4005,2.0455"
    lnk = []
    
    
    while True:
        
        
        # read the next frame from the file
        (grabbed, frame) = vs.read()
        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break
        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]
        # clone the output frame, then convert it from BGR to RGB
        # ordering, resize the frame to a fixed 224x224, and then
        # perform mean subtraction
        output = frame.copy()
        time.sleep(1)
        
        file_name = str(time.ctime())
        
        file_name = str(file_name[0:3]+"_"+file_name[11:13]+"-"+file_name[14:16]+"-"+file_name[17:19]+"_"+file_name[20:24])
        print(file_name)
        
        
       
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = image.img_to_array(frame)

        frame_tensor = np.expand_dims(frame, axis=0) 
        frame_tensor /= 255


        predict = new_model.predict(frame_tensor)
        predictions.append(predict)

        predict = predict*100
        predict = predict[0]
        maximum = max(predict)

        for i in range(0,len(predict)):
            if (predict[i] == maximum):
                print('c'+str(i) + "_" + classes_labels[i])
                text = 'c'+str(i) + "_" + classes_labels[i]
                
        if(text == 'c0_safe_driving'):
            cv2.putText(output, text, (40, 35), cv2.FONT_HERSHEY_SIMPLEX,
            1, (255, 0, 0), 2)
            cv2.imshow("output",output)
        else:
            if (flag == text):
                cv2.putText(output, text, (40, 35), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 0, 0), 2)
                cv2.imshow("output",output)
                
            else:
                flag = text
                cv2.putText(output, text, (40, 35), cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 0, 0), 2)
                cv2.imshow("output",output)
                cv2.imwrite('C:/Users/karth/Desktop/violations/vio_'+str(file_name)+'.jpg',output)
                lnk = img_2_link('C:/Users/karth/Desktop/violations/vio_'+str(file_name)+'.jpg')
                upload_data(Driver_id,gps,text[3:],lnk)
                
                
            

    
       


        key = cv2.waitKey(1) & 0xFF
        
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
        
        
        
        
        
        
        
        
        
        gps_full.append(gps)
        text_full.append(text)
    
    
    
    vs.release()
    cv2.destroyAllWindows()    

        
        
        
        

    return gps_full,text_full,Driver_id
     


    
   

In [8]:
gps_full,text_full,Driver_id = driver()

Tue_22-46-40_2020
c0_safe_driving
Tue_22-46-42_2020
c0_safe_driving
Tue_22-46-43_2020
c0_safe_driving
Tue_22-46-44_2020
c0_safe_driving
Tue_22-46-45_2020
c0_safe_driving
Tue_22-46-46_2020
c1_texting_left
ya
DRI_2 1.4005,2.0455 texting_left Hf4RRQTI!JZOSEIXV27xh2ZDoEP1_FAdzH5lK8RGAozr1IlJ-puk Uploaded
Tue_22-47-02_2020
c1_texting_left
Tue_22-47-03_2020
c1_texting_left
Tue_22-47-04_2020
c1_texting_left
Tue_22-47-05_2020
c1_texting_left
Tue_22-47-06_2020
c2_talking_on_the_phone_left
ya
DRI_2 1.4005,2.0455 talking_on_the_phone_left ea4lhYSJ!kF3KFiVHCYurIF9FzLdcN7V5jZmRaIvNIAq1IVBgyRo Uploaded
Tue_22-47-27_2020
c2_talking_on_the_phone_left
Tue_22-47-28_2020
c2_talking_on_the_phone_left
Tue_22-47-29_2020
c2_talking_on_the_phone_left
Tue_22-47-30_2020
c2_talking_on_the_phone_left
Tue_22-47-31_2020
c3_texting_right
ya
DRI_2 1.4005,2.0455 texting_right XO5nTIIK!daS4T2DWDbnmhmRlOatnHckyNIvQNd0IxIdNbXfWLeo Uploaded
Tue_22-47-49_2020
c3_texting_right
Tue_22-47-50_2020
c3_texting_right
Tue_22-47-51

In [9]:
print(gps_full)
print(text_full)

['1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455', '1.4005,2.0455']
['c0_safe_driving', 'c0_safe_driving', 'c0_safe_driving', 'c0_safe_driving', 'c0_safe_driving', 'c1_texting_left', 'c1_texting_left', 'c1_texting_left', 'c1_texting_left', 'c1_texting_left', 'c2_talking_on_the_phone_left', 'c2_talking_on_the_phone_left', 'c2_talking_on_the_phone_left'